In [1]:
from bs4 import BeautifulSoup as bs # for scraping the website
import requests 
import json # storing the information as a list of dictionaries
import re 
import time 

In [2]:
page_03 = bs(requests.get('http://insider.espn.com/nba/hollinger/statistics/_/year/2003', 'lxml').content)
table = page_03.select('table.tablehead tr')
columns = page_03.select('table.tablehead tr.colhead td')[0:14]
keys = [i.get_text() for i in columns]

#### Get the keys for the dictionaries that are going to be created, where keys are equal to column names in a Pandas Dataframe

In [3]:
base_urls = ['//insider.espn.com/nba/hollinger/statistics/_/year/' + str(i) for i in range(2003,2021)]
all_pages = []
def get_pages(url):
    page = bs(requests.get('http:' + url, 'lxml').content)
    all_pages.append(url)
    if '7' in page.select('div.page-numbers')[0].get_text():
        for i in range(2,8):
            next_page = '//insider.espn.com/nba/hollinger/statistics/_/page/' + str(i) + url[-10:]
            all_pages.append(next_page)
    elif '8' in page.select('div.page-numbers')[0].get_text():
        for i in range(2,9):
            next_page = '//insider.espn.com/nba/hollinger/statistics/_/page/' + str(i) + url[-10:]
            all_pages.append(next_page)
    return all_pages

#### Function  to help create a list of urls for all pages for all seasons from 2003-2020. 

In [4]:
for i in base_urls:
    get_pages(i)
all_pages[0:20]

['//insider.espn.com/nba/hollinger/statistics/_/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/page/2/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/page/3/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/page/4/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/page/5/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/page/6/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/page/7/year/2003',
 '//insider.espn.com/nba/hollinger/statistics/_/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/page/2/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/page/3/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/page/4/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/page/5/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/page/6/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/page/7/year/2004',
 '//insider.espn.com/nba/hollinger/statistics/_/year/2005',
 '//insider.espn

#### Passing in the function and checking to see if the links look correct, which they do.

In [5]:
def get_total_entries(url):
    results = int(bs(requests.get('http:' + url, 'lxml').content).select('div.totalResults')[0].get_text().split()[0])
    return results
total_entries = 0
for i in base_urls:
    total_entries += get_total_entries(i)
print(total_entries)

6115


#### Function to count the total number of entries (i.e. dictionaries) we should end up once scraping is complete.

In [6]:
def get_info(url):
    page = bs(requests.get('http:' + url, 'lxml').content)
    page_table = page.select('table.tablehead tr')
    del page_table[1::11]
    del page_table[0]
    player_raw = [i.select('td') for i in page_table]
    player_raw = [[j.get_text() for j in i] for i in player_raw]
    for i in range(len(player_raw)):
        master_data.append(dict(zip(keys,player_raw[i])))
    return master_data  

#### The main function which scrapes the pages and stores the stats for each player in a dictionary.

In [7]:
master_data=[]
for i in range(len(all_pages)):
    get_info(all_pages[i])
    time.sleep(0.1)

#### Passing the list of urls through the function to obtain player data.

In [8]:
print(len(master_data))
master_data[0:25]

6115


[{'RK': '1',
  'PLAYER': 'Tracy McGrady, ORL',
  'GP': '75',
  'MPG': '39.4',
  'TS%': '.564',
  'AST': '15.0',
  'TO': '7.1',
  'USG': '32.6',
  'ORR': '5.0',
  'DRR': '14.6',
  'REBR': '9.5',
  'PER': '31.01',
  'VA': '452.0',
  'EWA': '15.1'},
 {'RK': '2',
  'PLAYER': "Shaquille O'Neal, LAL",
  'GP': '67',
  'MPG': '37.8',
  'TS%': '.602',
  'AST': '10.7',
  'TO': '10.1',
  'USG': '27.8',
  'ORR': '11.0',
  'DRR': '21.6',
  'REBR': '16.5',
  'PER': '29.43',
  'VA': '356.2',
  'EWA': '11.9'},
 {'RK': '3',
  'PLAYER': 'Kobe Bryant, LAL',
  'GP': '82',
  'MPG': '41.5',
  'TS%': '.550',
  'AST': '16.0',
  'TO': '9.6',
  'USG': '31.1',
  'ORR': '3.0',
  'DRR': '15.3',
  'REBR': '9.3',
  'PER': '27.07',
  'VA': '420.6',
  'EWA': '14.0'},
 {'RK': '4',
  'PLAYER': 'Kevin Garnett, MIN',
  'GP': '82',
  'MPG': '40.5',
  'TS%': '.553',
  'AST': '20.4',
  'TO': '9.4',
  'USG': '25.0',
  'ORR': '9.0',
  'DRR': '28.5',
  'REBR': '18.8',
  'PER': '26.46',
  'VA': '370.8',
  'EWA': '12.4'},
 {'RK':

#### Looking at the data to make sure it is the same length as the ouput of the get_total_entires function, and to see if the data looks correct, which are both true, meaning we can now save this list.

In [9]:
def save_data(file,data):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
save_data('nba_player_stats.json', master_data)

#### This function save the list of dictionaries as a json file which can later be loaded in and easily transformed into a Pandas Dataframe for exploration and modelling.